# Abundances

`synthesizer` can be used to generate abundance patterns, in total and for gas and dust individually, for a given metallicity, alpha ehancement, and arbitrary element scalings.

At present this functionality is only utilised when creating cloudy input scripts. These scripts are used to calculate nebular line and continuum emission for a given incident spectral energy distribution and gas abundance pattern.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from synthesizer.abundances import (
    Abundances,
    abundance_scalings,
    depletion_models,
    plot_abundance_pattern,
    plot_multiple_abundance_patterns,
    reference_abundance_patterns,
)

by default initialising `Abundances` creates a solar abundance pattern with no depletion. The default solar abundance pattern is Asplund et al. (2009), though this can be changed if desired. 

In [ ]:
a0 = Abundances()

isinstance(Abundances, type)

like most `synthesizer` objects we can explore the important attributes of an object by using `print()`:

In [ ]:
print(a0)

You can access the logarithmic abundances ($\log_{10}(N_X/N_H)$) of an element like this:

In [ ]:
print(f"log10(O/H): {a0.total['O']:.2f}")
print(f"log10(O/H): {a0['O']:.2f}")

### Solar abundance pattern

As noted, there are several reference abundance patterns built into `synthesizer`. These can be accessed from `synthesizer.abundances.reference_abundance_patterns`:

In [ ]:
reference_abundance_patterns.available_patterns

In [ ]:
reference = reference_abundance_patterns.Asplund2009()
reference.ads
reference.abundance

In [ ]:
a1 = Abundances(reference=reference_abundance_patterns.Gutkin2016)
print(a1)

Solar abundance classes can also be called using a string representation of the name.

In [ ]:
a1 = Abundances(reference="Gutkin2016")
print(a1)

### Metallicity

We can specify a different metallicity. By default abundances are scaled from the Solar abundances provided through an optional argument (default Asplund et a. 2009). However, as we will see later, it is possible to set a different $\alpha$-enhancement or set arbitrary element scalings.

In [ ]:
a2 = Abundances(metallicity=0.01)
print(a2)

## $\alpha$-enhancement

We can also generate abundance patterns assuming different $\alpha$-enhancements. In this case it is necessary to re-scale the non-$\alpha$ elements to recover the input metallicity.

In [ ]:
a3 = Abundances(alpha=0.6)
print(a3)

We can print a relative solar abundance like this:

In [ ]:
print(f"[O/Fe] = {a3.reference_relative_abundance('O', ref_element='Fe'):.2f}")
print(f"[O/Fe] = {a3['[O/Fe]']:.2f}")
print(f"[B/Fe] = {a3['[B/Fe]']:.2f}")

That, is, as expected given that we set $\alpha=0.6$.

### Depletion

To account for metals being locked up in dust, we can also specify a depletion pattern. It is possible to either provide a dictionary of values or specify one of the in-built patterns. 

In [ ]:
# assume 99% of Carbon and Iron are depleted on to dust
depletion = {"C": 0.99, "Fe": 0.99}

# calculate the abundance patterns, now included gas and dust separately
a4 = Abundances(metallicity=0.01, depletion=depletion)
print(a4)

In [ ]:
a5 = Abundances(
    metallicity=0.0156,
    reference=reference_abundance_patterns.Gutkin2016,
    depletion_model=depletion_models.Gutkin2016,
)
print(a5)

In [ ]:
a6 = Abundances(
    metallicity=0.0156,
    reference=reference_abundance_patterns.Gutkin2016,
    depletion_model=depletion_models.CloudyClassic,
)
print(a6)

In [ ]:
a7 = Abundances(
    metallicity=0.0156,
    reference=reference_abundance_patterns.Gutkin2016,
    depletion_model=depletion_models.Jenkins2009_Gunasekera2021,
)
print(a7)

When specifying an in-built pattern it's also possible to specify an optional scaling parameter depending on the particular model.

Below we explore the effect of $F_{*}$ on the depletion factors for N, O, and S. 

In [ ]:
for element in ["N", "O", "S"]:
    print(element, "-" * 5)
    for fstar in [0.0, 0.5, 1.0]:
        depletion = depletion_models.Jenkins2009_Gunasekera2021(
            fstar
        ).depletion[element]
        print(f"{fstar} {depletion:.2f} {np.log10(depletion):.2f}")

Next, we explore the effect of $F_{*}$ on the dust mass fraction and dust-to-metal ratio ($\xi_{d}$):

In [ ]:
print("F_*", "dust_abundance", "dust_mass_fraction", "dust_to_metal_ratio")
for fstar in [0.0, 0.1, 0.25, 0.5, 1.0]:
    a = Abundances(
        metallicity=0.0156,
        reference=reference_abundance_patterns.Gutkin2016,
        depletion_model=depletion_models.Jenkins2009_Gunasekera2021,
        depletion_scale=fstar,
    )
    print(
        fstar,
        f"{a.dust_abundance:.2g}",
        f"{a.dust_mass_fraction:.2g}",
        f"{a.dust_to_metal_ratio:.2g}",
    )

We can also adjust the depletion scale in place and recalculate the dust mass:

In [ ]:
a = Abundances(
    metallicity=0.0156,
    reference=reference_abundance_patterns.Gutkin2016,
    depletion_model=depletion_models.Jenkins2009_Gunasekera2021,
    depletion_scale=0.5,
)

print(f"{a.dust_abundance:.2g}")

a.add_depletion(
    depletion_model="Jenkins2009_Gunasekera2021", depletion_scale=1.0
)

print(f"{a.dust_abundance:.2g}")

a.add_depletion(
    depletion_model="Jenkins2009_Gunasekera2021", depletion_scale=0.5
)

print(f"{a.dust_abundance:.2g}")

In [ ]:
# define default model
a = Abundances(
    metallicity=0.0156,
    reference=reference_abundance_patterns.Gutkin2016,
    depletion_model=depletion_models.Jenkins2009_Gunasekera2021,
    depletion_scale=0.5,
)


default_dust_mass_fraction = a.dust_mass_fraction

# vary fstar and recalaculate the dust_mass_fraction
for fstar in [0.0, 0.1, 0.25, 0.5, 1.0]:
    # recalcualte for different fstar
    a.add_depletion(
        depletion_model="Jenkins2009_Gunasekera2021", depletion_scale=fstar
    )

    dust_mass_fraction = a.dust_mass_fraction

    print(
        f"""{fstar} {default_dust_mass_fraction:.4f} {dust_mass_fraction:.4f}
        {dust_mass_fraction/default_dust_mass_fraction:.2f}"""
    )

When the depletion is applied the total, gas, and dust abundance patterns are provided e.g.

In [ ]:
print(f'log10(C_total/H) : {a7.total["C"]:.2f}')
print(f'log10(C_gas/H) : {a7.gas["C"]:.2f}')
print(f'log10(C_dust/H) : {a7.dust["C"]:.2f}')

### Arbitrary element scaling

We can also change the abundance of any specific element (or set of elements), with the abundances of other elements rescaled self-consistently to yield the correct metallicity. 

If the abundance is a float it is the logarithmic abundance ($\log_{10}(X/H)$) while if it is a string it is one of the in-built functions that scale the abundance with metallicity (e.g. the model proposed by Dopita et al. 2006). Note, combining this with a non-zero `alpha` can lead to a mild inconsistency.

Using a float for the absolute abundance (relative to H):

In [ ]:
a8 = Abundances(metallicity=0.0134, abundances={"nitrogen": -4.5})
print(a8)

Providing an abundance relative to another element:

In [ ]:
a9a = Abundances(metallicity=0.0134, abundances={"nitrogen_to_oxygen": -0.4})
print(a9a)

# same as above but less PEP8 compliant
a9b = Abundances(metallicity=0.0134, abundances={"N/O": -0.4})
print(a9b)

Alternatively, we can use a built-in function for particular set of elements or for all elements available. For example, here we use the Dopita (2006) scaling relation to adjust Nitrogen.

In [ ]:
a10 = Abundances(metallicity=0.0134, abundances={"nitrogen": "Dopita2006"})
print(a10)

We can also use a model to set all scalings for all available elements.

In [ ]:
a11 = Abundances(metallicity=0.015, abundances="GalacticConcordance")
print(a11)

##### Scaling models

Let's look in a bit more detail at some of the available scaling relationships.

We can also access the scaling functions directly:

In [ ]:
abundance_scalings.available_scalings

In [ ]:
abundance_scalings.GalacticConcordance().available_elements

In [ ]:
abundance_scalings.GalacticConcordance().nitrogen(0.016)

These functions also include useful meta data:

In [ ]:
print(abundance_scalings.GalacticConcordance().ads)
print(abundance_scalings.GalacticConcordance().doi)

Let's dig a little deeper and make a plot of X/O vs. O/H for one of these scalings. 

In [ ]:
scaling_study_name = "GalacticConcordance"

# create an array of metallicities equally space in log-space
log10metallicities = np.arange(-4.0, -1.5, 0.1)
metallicities = 10**log10metallicities

# the reference oxygen abundance for GalacticConcordance, i.e. [O/H]
reference_oxygen_abundance = -3.24
reference_metallicity = 0.015

# (O/H), assumed to scale linearly with metallicity
oxygen_abundance = reference_oxygen_abundance + np.log10(
    metallicities / reference_metallicity
)

scaling_study = getattr(abundance_scalings, scaling_study_name)()

for element, element_name in zip(
    scaling_study.available_elements, scaling_study.available_elements_names
):
    scaling = getattr(scaling_study, element_name)

    abundances = np.array(
        [scaling(metallicity) for metallicity in metallicities]
    )

    # log10(N/O) = log10(N/H) - log10(O/H)
    x_to_oxygen_ratios = abundances - oxygen_abundance

    plt.plot(oxygen_abundance + 12.0, x_to_oxygen_ratios, label=element)

plt.plot(
    [reference_oxygen_abundance + 12] * 2, [-6.0, -0.0], alpha=0.2, lw=2, c="k"
)

plt.ylabel(r"$\log_{10}(X/O)$")
plt.xlabel(r"$\log_{10}(O/H)+12$")
plt.legend()
plt.show()

## Plots

There are also a helper functions for plotting one or more abundance patterns, here we plot two abundance patterns with different alpha abundances:

In [ ]:
plot_multiple_abundance_patterns(
    [a2, a3],
    labels=[r"Z=0.01", r"Z=0.01; \alpha = 0.6"],
    show=True,
    ylim=[-7.0, -3.0],
)

We can plot the abundance pattern of each component:

In [ ]:
plot_abundance_pattern(
    a7, show=True, ylim=[-7.0, -3.0], components=["total", "gas", "dust"]
)